In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# 02_开始使用 Keras 函数式 API

Keras 函数式 API 是定义复杂模型（如多输出模型、有向无环图，或具有共享层的模型）的方法。

相较上一篇笔记，这里着重解释API的详细用法。

## 一切皆为张量：一个简单的例子--全连接网络

这个例子能够帮助我们对Keras API进行一些简单的理解。对于全连接网络，Sequential 模型可能是实现这种网络的一个更好选择。需要注意：

- 网络层的实例是可调用的，它以张量为参数，并且返回一个张量
- 输入和输出均为张量，它们都可以用来定义一个模型（Model）
- 这样的模型同 Keras 的 Sequential 模型一样，都可以被训练

In [1]:
# 这部分返回一个张量
inputs = Input(shape=(784,))

# 层的实例是可调用的，它以张量为参数，并且返回一个张量
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# 这部分创建了一个包含输入层和三个全连接层的模型
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # 开始训练


NameError: name 'Input' is not defined

注意这里的一个写法：x = Dense(64, activation='relu')(inputs)

在 Layer 这个类里面，有这样一个函数(https://github.com/keras-team/keras/blob/4cde148de0c37981c50f3a8e4a59fa4e5f653e17/keras/engine/topology.py#L541)：

def __call__(self, inputs, **kwargs):

通过定义这个函数，我们可以让一个对象可以当函数用。因此我们可以把下面的代码拆成这样：

在Python的官方文档中有详细说明(https://docs.python.org/3/reference/datamodel.html#emulating-callable-objects)

3.3.6. Emulating callable objects

object.__call__(self[, args...])

    Called when the instance is “called” as a function; if this method is defined, x(arg1, arg2, ...) is a shorthand for x.__call__(arg1, arg2, ...).

## 多输入多输出模型

以下是函数式 API 的一个很好的例子：具有多个输入和输出的模型。函数式 API 使处理大量交织的数据流变得容易。

来考虑下面的模型。我们试图预测 Twitter 上的一条新闻标题有多少转发和点赞数。模型的主要输入将是新闻标题本身，即一系列词语，但是为了增添趣味，我们的模型还添加了其他的辅助输入来接收额外的数据，例如新闻标题的发布的时间等。 该模型也将通过两个损失函数进行监督学习。较早地在模型中使用主损失函数，是深度学习模型的一个良好正则方法。

模型结构如下图所示：

![多输入多输出模型](https://s3.amazonaws.com/keras.io/img/multi-input-multi-output-graph.png)

让我们用函数式 API 来实现它。

主要输入接收新闻标题本身，即一个整数序列（每个整数编码一个词）。 这些整数在 1 到 10,000 之间（10,000 个词的词汇表），且序列长度为 100 个词。

In [2]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

# 标题输入：接收一个含有 100 个整数的序列，每个整数在 1 到 10000 之间。
# 注意我们可以通过传递一个 "name" 参数来命名任何层。
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# Embedding 层将输入序列编码为一个稠密向量的序列，
# 每个向量维度为 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# LSTM 层把向量序列转换成单个向量，
# 它包含整个序列的上下文信息
lstm_out = LSTM(32)(x)

print(keras.__version__)

from keras.layers import LST

Using TensorFlow backend.



2.2.4


在这里，我们插入辅助损失，使得即使在模型主损失很高的情况下，LSTM 层和 Embedding 层都能被平稳地训练。

In [2]:
auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

此时，我们将辅助输入数据与 LSTM 层的输出连接起来，输入到模型中：

In [3]:
auxiliary_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, auxiliary_input])

# 堆叠多个全连接网络层
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

# 最后添加主要的逻辑回归层
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

然后定义一个具有两个输入和两个输出的模型：

In [4]:
model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])

现在编译模型，并给辅助损失分配一个 0.2 的权重。如果要为不同的输出指定不同的 loss_weights 或 loss，可以使用列表或字典。 在这里，我们给 loss 参数传递单个损失函数，这个损失将用于所有的输出。

In [5]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy',
              loss_weights=[1., 0.2])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


我们可以通过传递输入数组和目标数组的列表来训练模型：

In [6]:
model.fit([headline_data, additional_data], [labels, labels],
          epochs=50, batch_size=32)

NameError: name 'headline_data' is not defined

由于输入和输出均被命名了（在定义时传递了一个 name 参数），我们也可以通过以下方式编译模型：

In [ ]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# 然后使用以下方式训练：
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': labels, 'aux_output': labels},
          epochs=50, batch_size=32)